# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [29]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import os

In [30]:
df_raw = pd.read_csv("/Users/scottonigman/Desktop/GA/homework/sonigman-dat-GA-HW/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [31]:
# frequency table for prestige and whether or not someone was admitted
df_prestige_freq = pd.crosstab(index=df["prestige"],columns=df["admit"])
df_prestige_freq

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [32]:
dummy_ranks = pd.get_dummies(df['prestige'],prefix='prestige')
print dummy_ranks.head()

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0             0             0             1             0
1             0             0             1             0
2             1             0             0             0
3             0             0             0             1
4             0             0             0             1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We only need 3, since we flattened the prestige variable to a binary, we are able to model the state of prestige by assuming the value of prestige when we know the status of 3/4 possible values. We do this to prevent multicollinearity in our model (if we used all 4 variables created from prestige their values would be correlated).

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [33]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [34]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
cross_prestige1= pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'], rownames=['admit'],margins=True)
print cross_prestige1

prestige_1.0    0   1  All
admit                     
0             243  28  271
1              93  33  126
All           336  61  397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [35]:
cross_prestige1.columns = ["0","1","rowtotal"]
cross_prestige1.index= ["0","1","coltotal"]
cross_prestige1/cross_prestige1.ix["coltotal"]
#The odds if you did attend a #1 level school are 54.0984%

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,rowtotal
0,0.723214,0.459016,0.68262
1,0.276786,0.540984,0.31738
coltotal,1.000000,1.000000,1.00000


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [36]:
cross_prestige1.columns = ["0","1","rowtotal"]
cross_prestige1.index= ["0","1","coltotal"]
cross_prestige1/cross_prestige1.loc["coltotal"]
#The odds if you did attend a #1 level school are 45.9016%

,0,1,rowtotal
0,0.723214,0.459016,0.68262
1,0.276786,0.540984,0.31738
coltotal,1.000000,1.000000,1.00000


#### 3.3 Calculate the odds ratio

In [37]:
#odds = p/1-p
admitted_p=0.540984
odds=admitted_p/(1-admitted_p)
print 1/odds

0.848483504133


#### 3.4 Write this finding in a sentenance: 

Answer: If you go to a prestige 1 class school, your odds of being admitted are .8484:1

#### 3.5 Print the cross tab for prestige_4

In [38]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
cross_prestige4= pd.crosstab(handCalc['admit'], handCalc['prestige_4.0'], rownames=['admit'],margins=True)
print cross_prestige4

prestige_4.0    0   1  All
admit                     
0             216  55  271
1             114  12  126
All           330  67  397


#### 3.6 Calculate the OR 

In [39]:
cross_prestige4.columns = ["0","1","rowtotal"]
cross_prestige4.index= ["0","1","coltotal"]
cross_prestige_perc=cross_prestige4/cross_prestige4.loc["coltotal"]
cross_prestige_perc
#The odds if you did attend a #4 level school are 17.91%

,0,1,rowtotal
0,0.654545,0.820896,0.68262
1,0.345455,0.179104,0.31738
coltotal,1.000000,1.000000,1.00000


#### 3.7 Write this finding in a sentence

The chances of being addmitted when one attended a class 4 prestige school are 17.91%

In [40]:
#odds = p/1-p
admitted_p4=cross_prestige_perc.loc['1','1']
odds4=admitted_p4/(1-admitted_p4)
print 1/odds4

4.58333333333


Answer: If you go to a prestige 4 class school, your odds of being admitted are 4.583:1

## Part 4. Analysis

In [41]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [42]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [43]:
  train_cols = data.columns[1:]

#### 4.2 Fit the model

In [44]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [45]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 28 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        20:57:59   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [46]:
params = result.params #subsetting the result parameters
conf = result.conf_int() #gets confidence intervervals from fit 
conf['OR'] = params #
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of being admitted when prestige_2 =1 (ie. one went to a prestige class 2 school) are 0.506:1

#### 4.6 Interpret the OR of GPA

Answer: For every 1 unit increase in GPA, the odds of being admitted increased by 2.18:1 and the odds ratio is 2.18:1 when one has the mean gpa

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [48]:
data_raw = pd.read_csv("/Users/scottonigman/Desktop/GA/homework/sonigman-dat-GA-HW/admissions.csv")
data = data_raw.dropna()
# instead of generating all possible values of GRE and GPA, we're going
  # to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas

#define the cartesian function
"""
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """
#cartesian product is every combination of possible values
#creating a cartesian product here to simulate each combination
#bottom part of function is a recursion 
#recursion so you can call a function using the previous results of the function, will change inputs to function
#this is a divide and conquer algorithm
#solves the problem again by splitting 
#every time you call function after 1st array you decrease the number of arrays -- building cartesian product from the end
# out and none logic is built as recursion occurs after first iteration
def cartesian(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype
    
    n = np.prod([x.size for x in arrays])
    print (n)
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
        
    m = n / arrays[0].size
    print (m)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [21]:
data.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


#### 5.1 Recreate the dummy variables

In [49]:
# recreate the dummy variables
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.columns = ['gre', 'gpa', 'prestige', 'intercept'] #vector creation before subsetting
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige') #split prestige to prevent collinearity
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':]) #re-join this

400
40
40
4
4
1
1
1


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [27]:
combos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
gre           400 non-null float64
gpa           400 non-null float64
intercept     400 non-null float64
prestige_2    400 non-null uint8
prestige_3    400 non-null uint8
prestige_4    400 non-null uint8
dtypes: float64(3), uint8(3)
memory usage: 10.6 KB


#### 5.2 Make predictions on the enumerated dataset

In [55]:
combos['admit_pred'] = result.predict(combos[train_cols])
result.summary()
print combos.tail(4)

       gre  gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0  \
396  800.0  4.0        1.0             0             0             0   
397  800.0  4.0        1.0             1             0             0   
398  800.0  4.0        1.0             0             1             0   
399  800.0  4.0        1.0             0             0             1   

     admit_pred  
396    0.734040  
397    0.582995  
398    0.419833  
399    0.368608  


#### 5.3 Interpret findings for the last 4 observations

Answer: The likelihood of being admitted for person 396 is 73%, for 397 it is 58%, for 398 it is 41.9%, and 36.8% for person 399.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.